In [24]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.document_loaders import CSVLoader
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains import RetrievalQA
from langchain_community.vectorstores import FAISS


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
llm = ChatGoogleGenerativeAI(model="models/gemini-2.5-flash", temperature=0.3)

In [4]:
loader = CSVLoader(file_path='D:/Repos/AIAgentProjects/Project2(ChatbotAgent)/FAQs.csv',source_column='prompt')
data = loader.load()

In [5]:
data[0]

Document(metadata={'source': 'Who are you? (1)', 'row': 0}, page_content='prompt: Who are you? (1)\nresponse: I am the virtual assistant designed to help visitors navigate the website.')

In [6]:
embeddings = HuggingFaceInstructEmbeddings(
    query_instruction="Represent the query for retrieval: "
)

C:\Users\Abc\AppData\Local\Temp\ipykernel_3788\3317292150.py:1: LangChainDeprecationWarning: The class `HuggingFaceInstructEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceInstructEmbeddings(
C:\Users\Abc\AppData\Local\Temp\ipykernel_3788\3317292150.py:1: LangChainDeprecationWarning: Default values for HuggingFaceInstructEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceInstructEmbeddings constructor instead.
  embeddings = HuggingFaceInstructEmbeddings(
No sentence-transformers model found with name hkunlp/instructor-large. Creating a new one with mean pooling.
c:\Users\Abc\AppData\Local\Programs\Python\Python314\Lib\site

In [7]:
e = embeddings.embed_query("What is LangChain? what is it used for?")

`SentenceTransformer._target_device` has been deprecated, please use `SentenceTransformer.device` instead.


In [12]:
instruct_embedding = HuggingFaceInstructEmbeddings()

C:\Users\Abc\AppData\Local\Temp\ipykernel_3788\2934677553.py:1: LangChainDeprecationWarning: Default values for HuggingFaceInstructEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceInstructEmbeddings constructor instead.
  instruct_embedding = HuggingFaceInstructEmbeddings()
No sentence-transformers model found with name hkunlp/instructor-large. Creating a new one with mean pooling.


In [15]:
vectorstore = FAISS.from_documents(data, instruct_embedding)

`SentenceTransformer._target_device` has been deprecated, please use `SentenceTransformer.device` instead.


In [16]:
retriever = vectorstore.as_retriever()

In [21]:
rdocs = retriever.invoke("is my data protected?")
rdocs

`SentenceTransformer._target_device` has been deprecated, please use `SentenceTransformer.device` instead.


[Document(id='4a40bb26-f42e-4cf1-938b-404bd48fc5e4', metadata={'source': 'Is my data secure? (264)', 'row': 263}, page_content='prompt: Is my data secure? (264)\nresponse: Yes your data is protected using industry standard security practices.'),
 Document(id='9d97f36f-2845-4b1a-8f98-e9592e30e736', metadata={'source': 'Is my data secure? (349)', 'row': 348}, page_content='prompt: Is my data secure? (349)\nresponse: Yes your data is protected using industry standard security practices.'),
 Document(id='eb87726f-2731-480c-ada5-7e4791bab95e', metadata={'source': 'Is my data secure? (179)', 'row': 178}, page_content='prompt: Is my data secure? (179)\nresponse: Yes your data is protected using industry standard security practices.'),
 Document(id='f5b5c600-aa60-42dd-9bd8-d08c41d434b9', metadata={'source': 'Is my data secure? (119)', 'row': 118}, page_content='prompt: Is my data secure? (119)\nresponse: Yes your data is protected using industry standard security practices.')]

In [25]:
RetrievalQA_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    input_key="query",
    return_source_documents=True
)

In [26]:
RetrievalQA_chain("is my data protected?")

C:\Users\Abc\AppData\Local\Temp\ipykernel_3788\1047453083.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  RetrievalQA_chain("is my data protected?")
`SentenceTransformer._target_device` has been deprecated, please use `SentenceTransformer.device` instead.


{'query': 'is my data protected?',
 'result': 'Yes your data is protected using industry standard security practices.',
 'source_documents': [Document(id='4a40bb26-f42e-4cf1-938b-404bd48fc5e4', metadata={'source': 'Is my data secure? (264)', 'row': 263}, page_content='prompt: Is my data secure? (264)\nresponse: Yes your data is protected using industry standard security practices.'),
  Document(id='9d97f36f-2845-4b1a-8f98-e9592e30e736', metadata={'source': 'Is my data secure? (349)', 'row': 348}, page_content='prompt: Is my data secure? (349)\nresponse: Yes your data is protected using industry standard security practices.'),
  Document(id='eb87726f-2731-480c-ada5-7e4791bab95e', metadata={'source': 'Is my data secure? (179)', 'row': 178}, page_content='prompt: Is my data secure? (179)\nresponse: Yes your data is protected using industry standard security practices.'),
  Document(id='f5b5c600-aa60-42dd-9bd8-d08c41d434b9', metadata={'source': 'Is my data secure? (119)', 'row': 118}, page

In [27]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's question based on the retrieved context:\n\n{context}"),
    ("human", "{input}")
])

combine_docs_chain = create_stuff_documents_chain(llm, prompt)
RetrievalQA_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [33]:
response = RetrievalQA_chain.invoke({
    "input": "Is my data protected?"
})

`SentenceTransformer._target_device` has been deprecated, please use `SentenceTransformer.device` instead.


In [34]:
response

{'input': 'Is my data protected?',
 'context': [Document(id='0569654d-31fd-445d-ac72-ebd80b50b285', metadata={'source': 'Is my data secure? (129)', 'row': 128}, page_content='prompt: Is my data secure? (129)\nresponse: Yes your data is protected using industry standard security practices.'),
  Document(id='1bd8861a-c776-4f5c-b1da-3eaccdcf5f88', metadata={'source': 'Is my data secure? (249)', 'row': 248}, page_content='prompt: Is my data secure? (249)\nresponse: Yes your data is protected using industry standard security practices.'),
  Document(id='4a40bb26-f42e-4cf1-938b-404bd48fc5e4', metadata={'source': 'Is my data secure? (264)', 'row': 263}, page_content='prompt: Is my data secure? (264)\nresponse: Yes your data is protected using industry standard security practices.'),
  Document(id='9d97f36f-2845-4b1a-8f98-e9592e30e736', metadata={'source': 'Is my data secure? (349)', 'row': 348}, page_content='prompt: Is my data secure? (349)\nresponse: Yes your data is protected using industr